# Exercise 1: reading a table and converting coordinates/times (10 points)

1. Download the file `fermi_lat_grbs.dat` from today’s exercise page on the website. It contains 1000 gamma-ray bursts observed by the Fermi satellite between 2010 and 2018.

2. Use the `astropy.io.ascii.read()` routine to read the file into a table. You need to specify the `header_start`, `data_start`, and `delimiter` arguments.

3. Display the column headers using the `columns` or `colnames` method.

4. Notice that RA and DEC are in sexagesimal format, represented as strings. Use `astropy.coordinates.Angle` to convert them to decimal degrees.  Print the table.

5. Do some brief analysis:  
  a) Construct a boolean mask array that selects only those bursts with durations $t_{90} < 2$ seconds and relative errors in the duration $\frac{t_{90\_error}}{t_{90}} < 50\%$.  
  b) Construct a second boolean mask array that selects $t_{90} > 2$ seconds with the same relative error.  
  c) Compare the median peak energy ($f_{lnc\_band\_epeak}$) of the two samples. Use the NumPy `median` function to compute the values.


In [2]:
#Code Here
import numpy as np
import astropy
from astropy import units as units
from astropy.table import Table
from astropy import coordinates
from astropy import units as u
# Q.1,2
data = astropy.io.ascii.read(r"C:\Users\eklav\OneDrive - University of Illinois - Urbana\astro_310\labs\lab14\fermi_lat_grbs.dat",format='basic', delimiter='|', guess=False, header_start = 2, data_start = 3)
# Q.3
print(data.columns)
# Q.4
data['ra'] = coordinates.Angle(data['ra'],unit = u.hour)

data['dec'] = coordinates.Angle(data['dec'],unit = u.deg)

data['ra'] = (data['ra']).to(u.deg)
data['dec'] = (data['dec']).to(u.deg)

print(data['ra'])
print(data['dec'])

# Q 5
mask_short_bursts = (data['t90'] < 2) & ((data['t90_error'] / data['t90']) < 0.5)
mask_long_bursts = (data['t90'] > 2) & ((data['t90_error'] / data['t90']) < 0.5)
short_bursts_epeak = data['flnc_band_epeak'][mask_short_bursts]
long_bursts_epeak = data['flnc_band_epeak'][mask_long_bursts]
median_peak_energy_short = np.median(short_bursts_epeak)
median_peak_energy_long = np.median(long_bursts_epeak)



<TableColumns names=('col0','name','ra','dec','trigger_time','t90','t90_error','t90_start','fluence','fluence_error','flux_1024','flux_1024_error','flux_1024_time','flux_64','flux_64_error','flnc_band_ampl','flnc_band_ampl_pos_err','flnc_band_ampl_neg_err','flnc_band_epeak','flnc_band_epeak_pos_err','flnc_band_epeak_neg_err','flnc_band_alpha','flnc_band_alpha_pos_err','flnc_band_alpha_neg_err','flnc_band_beta','flnc_band_beta_pos_err','flnc_band_beta_neg_err','flnc_spectrum_start','flnc_spectrum_stop','pflx_best_fitting_model','pflx_best_model_redchisq','flnc_best_fitting_model','flnc_best_model_redchisq','_1')>
        ra        
       deg        
------------------
337.86999999999995
356.95916666666665
211.21999999999997
            209.14
2.8999999999999995
 9.479999999999999
244.03999999999996
20.722916666666666
 311.3299999999999
160.42999999999998
               ...
            308.56
235.66999999999996
159.91083333333333
              52.3
202.26999999999995
 304.9599999999999


C:\Users\eklav\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [3]:
median_peak_energy_short, median_peak_energy_long

(359.3643, 135.48020000000002)

# Exercise 2: working with and modifying a table (10 points)

Download the files "mcxc.dat" and "mcxc.readme" from today’s exercise page on the website. These files contain a catalog of X-ray-detected clusters of galaxies from Piffaretti et al. (2011) obtained through the VizieR service at the University of Strasbourg.

1. Use `astropy.io.ascii.read()` to read the table and its metadata into Python. This table is in "CDS" format, and you specify the metadata file using the `readme` argument.

2. Extract the log of L500 (luminosity in units of $10^{44} \text{erg/s}$), log of M500 (mass in $10^{14}$ solar masses), and z (redshift) columns from the data into 1D arrays. Create a mask array selecting those clusters with redshift $< 0.1$.

3. Now construct the array $r$ containing $r = \log L - 1.64 \log M$. The X-ray luminosity and mass of galaxy clusters are correlated roughly such that $L \propto M^{5/3}$, so the range in $r$ is small.  
  
4. Modify the table to add a masked column (`MaskedColumn` object) for $r$, and use the description "$r = \log L - 1.64 \log M$" for that column. Use the mask array you created to mask those clusters you don't want to store $r$ values for. Print the table to check whether your column was added correctly. You should see "--" for masked-out rows.  

5. Write the modified table to a file named "mcxc_new.csv" in comma-separated value (CSV) format.  




In [5]:
#Code Here
from astropy.table import Table, MaskedColumn
data = astropy.io.ascii.read(r"C:\Users\eklav\OneDrive - University of Illinois - Urbana\astro_310\labs\lab14\mcxc.dat", readme="mcxc.readme", format="cds")

l500 = data['L500'].data
M500 = data['M500'].data
z = data['z'].data
mask = z < 0.1
filter_l500 = l500[mask]
filter_M500 = M500[mask]
filtered_z = z[mask]
r = filter_l500 - 1.64 * filter_M500
print(r)
r = data['L500'] - 1.64 * data['M500']
mask = data['z'] >= 0.1
r_column = MaskedColumn(r, mask=mask, name='r', description='r = log L - 1.64 log M')


[-1.012892 -0.48837  -1.528762 -1.997207 -1.614681 -1.169538 -2.294202
 -2.34182  -1.294671 -1.260939 -1.572374 -1.259829 -1.815282 -1.163526
 -2.483263 -2.112338 -1.616338 -2.366497 -2.387405 -0.699704 -1.025122
 -1.309052 -0.592733 -2.145387 -3.618647 -2.754706 -1.25405  -2.210964
 -2.621285 -1.921318 -1.446881 -1.106659 -2.60272  -1.785156 -0.581953
 -2.558346 -0.998769 -1.604541 -1.954547 -0.983408 -0.593058 -0.32764
 -1.021002 -0.178581 -1.577728 -0.715587 -0.590573 -1.981609 -0.534731
 -0.860869 -0.043123 -2.062197 -0.5445   -0.403451 -0.583102 -1.208672
 -0.183307 -1.104538 -0.645966 -1.256982 -0.876203 -1.530939 -0.069357
 -2.206527 -2.042895 -1.776408 -0.426818 -1.00719  -0.865692 -1.663936
 -1.736471 -1.315752 -1.438803 -0.918986 -1.672694 -1.453302 -0.356023
 -1.798011 -1.379327 -1.92645  -0.648112 -0.673878 -2.154409 -0.482401
 -1.09285  -3.369212 -3.503389 -2.219902 -1.219815 -0.930073 -0.432375
 -1.306341 -1.331942 -2.218432 -3.391544 -3.869953 -1.255464 -1.508218
 -1.741

In [6]:
data.add_column(r_column)
print(data)
data.write(r'C:\Users\eklav\OneDrive - University of Illinois - Urbana\astro_310\labs\lab14\mcxc_new.csv', format='csv', overwrite=True)

    MCXC          OName             AName        ... L500r4          r          
                                                 ...              1e+37 W       
------------ ---------------- ------------------ ... ------ --------------------
J0000.1+0816 RXC J0000.1+0816          UGC 12890 ...     --            -1.012892
J0000.4-0237 RXC J0000.4-0237                 -- ...     -- -0.48836999999999997
J0001.6-1540 RXC J0001.6-1540                 -- ...     --                   --
J0001.9+1204 RXC J0001.9+1204              A2692 ...     --                   --
J0003.1-0605  RXCJ0003.1-0605              A2697 ...     --                   --
J0003.2-3555  RXCJ0003.2-3555              A2717 ...     --            -1.528762
J0003.8+0203  RXCJ0003.8+0203              A2700 ...     --  -1.9972069999999997
J0004.9+1142 RXC J0004.9+1142          UGC 00032 ...     --            -1.614681
J0005.3+1612 RXC J0005.3+1612              A2703 ...     --                   --
J0006.0-3443  RXCJ0006.0-344

: 